In [7]:
# how to set to GPU***
import learn as learn
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import wandb.wandb_torch as torch
import fastai.vision.all
import sys
import timm
import torch.nn.functional as tnnf
from glob import glob
from os import walk

In [8]:
def file_loader():
    path = 'petfinder-pawpularity-score/'

    train_df = pd.read_csv(path + 'train.csv')
    test_df = pd.read_csv(path + 'test.csv')
    train_jpg = glob(path + "train/*.jpg")
    test_jpg = glob(path + "test/*.jpg")

    print('test_jpg is of type ', type(test_jpg[0]), ' and length ', len(test_jpg))
    print(test_jpg[0])
    return train_df, test_df, train_jpg, train_df, path

In [13]:
def data_loader(train_df, test_df, train_jpg, test_jpg, dataset_path):
    fastai.vision.all.set_seed(999, reproducible=True)
    train_df['path'] = train_df['Id'].map(lambda x: str(dataset_path + 'train/' + x) + '.jpg')
    train_df = train_df.drop(columns=['Id'])
    train_df = train_df.sample(frac=1).reset_index(drop=True)  # shuffle dataframe
    print(train_df.info(max_cols=train_df.shape[1]))
    train_df['Pawpularity'].hist(figsize=(10, 5))
    print(f"The mean Pawpularity score is {train_df['Pawpularity'].mean()}")
    print(f"The median Pawpularity score is {train_df['Pawpularity'].median()}")
    print(f"The standard deviation of the Pawpularity score is {train_df['Pawpularity'].std()}")
    print(f"There are {len(train_df['Pawpularity'].unique())} unique values of Pawpularity score")
    train_df['norm_score'] = train_df['Pawpularity'] / 100
    print(train_df['norm_score'])
    im = fastai.vision.all.Image.open(train_df['path'][1])
    width, height = im.size
    print(width, height)
    dls2 = fastai.vision.all.ImageDataLoaders.from_df(train_df,  # pass in train DataFrame
                                                      valid_pct=0.2,  # 80-20 train-validation random split
                                                      seed=999,  # seed
                                                      fn_col='path',
                                                      # filename/path is in the second column of the
                                                      # DataFrame
                                                      label_col='norm_score',
                                                      # label is in the first column of the DataFrame
                                                      y_block=fastai.vision.all.RegressionBlock,
                                                      # The type of target
                                                      bs=32,  # pass in batch size
                                                      num_workers=8,
                                                      item_tfms=fastai.vision.all.Resize(224),
                                                      # pass in item_tfms
                                                      batch_tfms=fastai.vision.all.setup_aug_tfms(
                                                          [fastai.vision.all.Brightness(),
                                                           fastai.vision.all.Contrast(),
                                                           fastai.vision.all.Hue(),
                                                           fastai.vision.all.Saturation()]))
    dls2.show_batch()
    return dls2

In [10]:

def petfinder_rmse(input, target):
    return 100 * torch.sqrt(tnnf.mse_loss(tnnf.sigmoid(input.flatten()), target))


In [11]:
def learner(dls, model):
    learn = Learner(dls, model, loss_func=BCEWithLogitsLossFlat(), metrics=petfinder_rmse).to_fp16()
    learn.lr_find(end_lr=3e-2)
    learn.fit_one_cycle(10, 2e-5, cbs=[SaveModelCallback(),
                                       EarlyStoppingCallback(monitor='petfinder_rmse', comp=np.less, patience=3)])
    learn.recorder.plot_loss()
    learn = learn.to_fp32()
    learn.save('fine-tuned')
    learn.export()



In [14]:
from fastai.learner import Learner
from fastai.callback.tracker import SaveModelCallback

from fastai.callback.tracker import EarlyStoppingCallback


def main():
    if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
        os.makedirs('/root/.cache/torch/hub/checkpoints/')
    !cp '../input/swin-transformer/swin_large_patch4_window7_224_22kto1k.pth' '/root/.cache/torch/hub/checkpoints/swin_large_patch4_window7_224_22kto1k.pth'
    train_df, test_df, train_jpg, train_df, path = file_loader()
    dls = data_loader(train_df, test_df, train_jpg, train_df, path)
    health = timm.create_model('swin_large_patch4_window7_224', pretrained=True, num_classes=dls.c)
    learner(dls, health)




